## Configuração do ambiente

#### Instalação do Pyspark e conexão com Google Drive

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 62.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=32083e7d9c54b5f94d2e081403946de8a301e6ada29cde27565970bd0868469f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("monkeypoxcolab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
spark

#### Importando as bibliotecas utilizadas no projeto

In [7]:
!pip install -U plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 5.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [8]:
from pyspark.sql import types as typ
from pyspark.sql import functions as f
from pyspark.sql.functions import expr, regexp_replace, desc
from pyspark.sql.functions import col,lit,create_map, collect_list
from pyspark.sql.functions import col, isnan,when, count, countDistinct
from IPython.core.display import HTML

In [9]:
import numpy as np
import pandas as pd
import plotly.express as px

#### Importando dataset global sobre a Varíola dos macacos. Atualizado até 23/09/2022.
##### Fonte: World in Data - Data Health Center

In [10]:
monkeypoxdf = spark.read.csv("/content/drive/MyDrive/Monkey_Pox_Dataset/latest_deprecated.csv", sep=",", header=True, inferSchema=True)

## Análise exploratória do dataframe

### Panorama geral do Dataframe

In [11]:
col = len(monkeypoxdf.columns)
col

36

In [12]:
row = monkeypoxdf.count()
row

69639

In [13]:
print(f'A dimensáo do Dataframe é: {(row,col)}')
print(f'O Dataframe possui o total de {row} linhas e {col} colunas')

A dimensáo do Dataframe é: (69639, 36)
O Dataframe possui o total de 69639 linhas e 36 colunas


In [14]:
monkeypoxdf.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country_ISO3: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Date_onset: timestamp (nullable = true)
 |-- Date_confirmation: timestamp (nullable = true)
 |-- Symptoms: string (nullable = true)
 |-- Hospitalised (Y/N/NA): string (nullable = true)
 |-- Date_hospitalisation: timestamp (nullable = true)
 |-- Isolated (Y/N/NA): string (nullable = true)
 |-- Date_isolation: timestamp (nullable = true)
 |-- Outcome: string (nullable = true)
 |-- Contact_comment: string (nullable = true)
 |-- Contact_ID: integer (nullable = true)
 |-- Contact_location: string (nullable = true)
 |-- Travel_history (Y/N/NA): string (nullable = true)
 |-- Travel_history_entry: string (nullable = true)
 |-- Travel_history_start: string (nullable = true)
 |-- Travel_

In [16]:
monkeypoxdf.agg(
    f.count('Status').alias('count_Status'),
    f.countDistinct('Status').alias('distinct_Status'),
    f.count('Location').alias('count_Location'),
    f.countDistinct('Location').alias('distinct_Location'),
    f.count('City').alias('count_City'),
    f.countDistinct('City').alias('distinct_City'),
    f.count('Country').alias('count_Country'),
    f.countDistinct('Country').alias('distinct_Country'),
    f.count('Country_ISO3').alias('count_Country_ISO3'),
    f.countDistinct('Country_ISO3').alias('distinct_Country_ISO3'),
    f.count('Age').alias('count_Age'),
    f.countDistinct('Age').alias('distinct_Age'),
    f.count('Gender').alias('count_Gender'),
    f.countDistinct('Gender').alias('distinct_Gender'),
    f.count('Date_onset').alias('Date_onset'),
    f.countDistinct('Date_onset').alias('distinct_Date_onset'),
    f.count('Date_confirmation').alias('count_Date_confirmation'),
    f.countDistinct('Date_confirmation').alias('distinct_Date_confirmation'),
    f.count('Symptoms').alias('count_Symptoms'),
    f.countDistinct('Symptoms').alias('distinct_Symptoms'),
    f.count('Hospitalised (Y/N/NA)').alias('count_Hospitalised'),
    f.countDistinct('Hospitalised (Y/N/NA)').alias('distinct_Hospitalised'),
    f.count('Date_hospitalisation').alias('count_Date_hospitalisation'),
    f.countDistinct('Date_hospitalisation').alias('distinct_Date_hospitalisation'),
    f.count('Isolated (Y/N/NA)').alias('count_Isolated'),
    f.countDistinct('Isolated (Y/N/NA)').alias('distinct_Isolated'),
    f.count('Date_isolation').alias('count_Date_isolation'),
    f.countDistinct('Date_isolation').alias('distinct_Date_isolation'),
    f.count('Outcome').alias('count_Outcome'),
    f.countDistinct('Outcome').alias('distinct_Outcome'),
    f.count('Contact_comment').alias('count_Contact_comment'),
    f.countDistinct('Contact_comment').alias('distinct_Contact_comment'),
    f.count('Contact_ID').alias('count_Contact_ID'),
    f.countDistinct('Contact_ID').alias('distinct_Contact_ID'),
    f.count('Contact_location').alias('count_Contact_location'),
    f.countDistinct('Contact_location').alias('distinct_Contact_location'),
    f.count('Travel_history (Y/N/NA)').alias('count_Travel_history'),
    f.countDistinct('Travel_history (Y/N/NA)').alias('distinct_Travel_history'),
    f.count('Travel_history_entry').alias('count_Travel_history_entry'),
    f.countDistinct('Travel_history_entry').alias('distinct_Travel_history_entry'),
    f.count('Travel_history_start').alias('count_Travel_history_start'),
    f.countDistinct('Travel_history_start').alias('distinct_Travel_history_start'),
    f.count('Travel_history_location').alias('count_Travel_history_location'),
    f.countDistinct('Travel_history_location').alias('distinct_Travel_history_location'),
    f.count('Travel_history_country').alias('count_Travel_history_country'),
    f.countDistinct('Travel_history_country').alias('distinct_Travel_history_country'),
    f.count('Genomics_Metadata').alias('count_Genomics_Metadata'),
    f.countDistinct('Genomics_Metadata').alias('distinct_Genomics_Metadata'),
    f.count('Confirmation_method').alias('count_Confirmation_method'),
    f.countDistinct('Confirmation_method').alias('distinct_Confirmation_method'),
    f.count('Source').alias('count_Source'),
    f.countDistinct('Source').alias('distinct_Source'),
    f.count('Source_II').alias('count_Source_II'),
    f.countDistinct('Source_II').alias('distinct_Source_II'),
    f.count('Source_III').alias('count_Source_III'),
    f.countDistinct('Source_III').alias('distinct_Source_III'),
    f.count('Source_IV').alias('count_Source_IV'),
    f.countDistinct('Source_IV').alias('distinct_Source_IV'),
    f.count('Source_V').alias('count_Source_V'),
    f.countDistinct('Source_V').alias('distinct_Source_V'),
    f.count('Source_VI').alias('count_Source_VI'),
    f.countDistinct('Source_VI').alias('distinct_Source_VI'),
    f.count('Source_VII').alias('count_Source_VII'),
    f.countDistinct('Source_VII').alias('distinct_Source_VII'),
    f.count('Date_entry').alias('count_Date_entry'),
    f.countDistinct('Date_entry').alias('distinct_Date_entry'),
    f.count('Date_death').alias('count_Date_death'),
    f.countDistinct('Date_death').alias('distinct_Date_death'),
    f.count('Date_last_modified').alias('count_Date_last_modified'),
    f.countDistinct('Date_last_modified').alias('distinct_Date_last_modified')
).show()

+------------+---------------+--------------+-----------------+----------+-------------+-------------+----------------+------------------+---------------------+---------+------------+------------+---------------+----------+-------------------+-----------------------+--------------------------+--------------+-----------------+------------------+---------------------+--------------------------+-----------------------------+--------------+-----------------+--------------------+-----------------------+-------------+----------------+---------------------+------------------------+----------------+-------------------+----------------------+-------------------------+--------------------+-----------------------+--------------------------+-----------------------------+--------------------------+-----------------------------+-----------------------------+--------------------------------+----------------------------+-------------------------------+-----------------------+--------------------------

In [17]:
monkeypoxdf.agg(
    f.count('Status').alias('Status_count'),
    f.countDistinct('Status').alias('Status_distinct'),
    f.count('Country').alias('Country_count'),
    f.countDistinct('Country').alias('Country_distinct'),
    f.count('Age').alias('Age_count'),
    f.countDistinct('Age').alias('Age_distinct'), 
    f.count('Gender').alias('Gender_count'),
    f.countDistinct('Gender').alias('Gender_distinct'),
    f.count('Symptoms').alias('Symptoms_count'),
    f.countDistinct('Symptoms').alias('Symptoms_distinct')
).show()

+------------+---------------+-------------+----------------+---------+------------+------------+---------------+--------------+-----------------+
|Status_count|Status_distinct|Country_count|Country_distinct|Age_count|Age_distinct|Gender_count|Gender_distinct|Symptoms_count|Symptoms_distinct|
+------------+---------------+-------------+----------------+---------+------------+------------+---------------+--------------+-----------------+
|       69595|              4|        69595|             129|     3021|          95|        2475|              6|           220|              112|
+------------+---------------+-------------+----------------+---------+------------+------------+---------------+--------------+-----------------+



In [18]:
monkeypoxdf.groupBy("Status").agg(countDistinct('Country')) \
    .show(truncate=False)

+----------+-----------------------+
|Status    |count(DISTINCT Country)|
+----------+-----------------------+
|omit_error|15                     |
|null      |0                      |
|suspected |19                     |
|confirmed |112                    |
|discarded |33                     |
+----------+-----------------------+



In [19]:
monkeypoxdf.groupBy("Status").agg(countDistinct('Age')) \
    .show(truncate=False)

+----------+-------------------+
|Status    |count(DISTINCT Age)|
+----------+-------------------+
|omit_error|2                  |
|null      |0                  |
|suspected |4                  |
|confirmed |92                 |
|discarded |15                 |
+----------+-------------------+



In [20]:
monkeypoxdf.groupBy("Status").agg(countDistinct('Gender')) \
    .show(truncate=False)

+----------+----------------------+
|Status    |count(DISTINCT Gender)|
+----------+----------------------+
|omit_error|1                     |
|null      |0                     |
|suspected |2                     |
|confirmed |5                     |
|discarded |3                     |
+----------+----------------------+



#### Explorando a coluna "Age"

##### Lista com todas as faixas etárias registradas no Dataframe

In [21]:
monkeypoxdf.select(collect_list("Age")).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
monkeypoxdf.groupBy("Age").agg(countDistinct('Country')) \
    .show(96)

+-----+-----------------------+
|  Age|count(DISTINCT Country)|
+-----+-----------------------+
|  5-9|                      2|
|20-40|                      1|
|50-55|                      1|
|20-24|                     19|
|20-59|                      4|
|15-69|                      1|
|45-54|                      1|
|29-43|                      1|
|51-65|                      1|
|20-69|                      4|
|30-50|                      1|
|23-46|                      1|
|25-39|                      2|
|65-69|                      4|
|41-45|                      1|
|15-39|                      1|
|30-39|                     10|
|25-44|                      1|
|20-39|                      2|
| null|                    111|
|  0-9|                      1|
|40-45|                      2|
| 0-49|                      2|
|30-49|                      2|
|22-59|                      1|
| 0-59|                      1|
|20-29|                      5|
|45-50|                      1|
|55-64| 

##### Lista com todos os sintomas registrados no Dataframe

In [23]:
monkeypoxdf.select(collect_list("Symptoms")).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
monkeypoxdf.groupBy("Symptoms").agg(countDistinct('Country')) \
    .show(122)

+--------------------+-----------------------+
|            Symptoms|count(DISTINCT Country)|
+--------------------+-----------------------+
|headache, skin le...|                      1|
|Perianal rash, fever|                      1|
|Fever, chills, fa...|                      1|
|headache, muscle ...|                      1|
|fever, general ma...|                      1|
|fever, outbreak o...|                      1|
|                rash|                      2|
|rash, body pains,...|                      1|
|genital ulcer les...|                      1|
|             lesions|                      1|
|characteristic sy...|                      1|
| skin manifestations|                      1|
|      fever, lesions|                      2|
|symptoms compatib...|                      1|
|headache, muscle ...|                      1|
|                Rash|                      1|
|      fever; myalgia|                      1|
|Spots on skin, ve...|                      1|
|         ski

#### Rankings presentes nas colunas mais importantes do dataframe

##### Ranking dos 20 países com maior número de casos registrados

In [84]:
monkeypoxdf.createOrReplaceTempView("MonkeypoxRank1")
spark.sql("select Country, count(Status) as count_Status from MonkeypoxRank1 " +
          "group by Country having count_Status >= 1 "
          "order by count_Status desc").show(20)

+--------------------+------------+
|             Country|count_Status|
+--------------------+------------+
|       United States|       24663|
|              Brazil|        7347|
|               Spain|        7269|
|              France|        3961|
|             Germany|        3590|
|             England|        3412|
|Democratic Republ...|        3048|
|                Peru|        2254|
|            Colombia|        1653|
|              Canada|        1488|
|              Mexico|        1369|
|         Netherlands|        1221|
|            Portugal|         908|
|               Italy|         846|
|               Chile|         787|
|             Belgium|         757|
|             Nigeria|         704|
|         Switzerland|         503|
|               Ghana|         402|
|           Argentina|         328|
+--------------------+------------+
only showing top 20 rows



##### Ranking das 20 faixas de idade com mais casos registrados

###### Faixa etária adotada seguindo a prática classificaçao etária em estudos científicos com dados de doenças x idade em datasets que apresentam inconsistência na catalogação dos registros da idade de pacientes

In [26]:
monkeypoxFaixa = spark.createDataFrame([("Crianças","0 a 12 anos"),("Adolescentes","13 a 19 anos"),("Adultos","20 a 95 anos")],["Classificação","Faixa etária"])
monkeypoxFaixa.show()

monkeypoxdf.createOrReplaceTempView("MonkeypoxAgeRank")
spark.sql("select Age, count(Country) as count_Age from MonkeypoxAgeRank " +
          "group by Age having count_Age >= 1 " + 
          "order by count_Age desc").show(20)

+-------------+------------+
|Classificação|Faixa etária|
+-------------+------------+
|     Crianças| 0 a 12 anos|
| Adolescentes|13 a 19 anos|
|      Adultos|20 a 95 anos|
+-------------+------------+

+-----+---------+
|  Age|count_Age|
+-----+---------+
| null|    66574|
|20-69|      616|
|15-64|      275|
|20-59|      244|
|15-74|      240|
|20-64|      225|
|15-84|      187|
|15-69|      184|
|30-34|       75|
|20-44|       60|
|25-29|       58|
|20-99|       57|
| 0-69|       54|
|35-39|       46|
|18-61|       44|
|40-44|       42|
| 1-69|       37|
|19-59|       36|
| 0-59|       34|
|23-50|       32|
+-----+---------+
only showing top 20 rows



In [27]:
monkeypoxAgedf=spark.createDataFrame([("1","null","null",66574),("2","20-69","adulto",616),("3","15-64","indeterminado",275),("4","15-64","indeterminado",275),("5","20-59","adulto",244),("6","15-74","indeterminado",240),("7","20-64","adulto",225), ("8","15-84","indeterminado",187),("9","15-69","indeterminado",184),("10","30-34","adulto",75),("11","20-44","adulto",60),("12","25-29","adulto",58),("13","20-99","adulto",57),("14","0-69","indeterminado",54),("15","35-39","adulto",46),("16","18-61","indeterminado",44),("17","40-44","adulto",42),("18","1-69","indeterminado",37),("19","19-59","indeterminado",36),("20","0-59","indeterminado",34),("21","23-50","adulto",32)],["ID","Faixa_etaria","Classificacao","Quantidade"])
monkeypoxAgedf.show(20)

+---+------------+-------------+----------+
| ID|Faixa_etaria|Classificacao|Quantidade|
+---+------------+-------------+----------+
|  1|        null|         null|     66574|
|  2|       20-69|       adulto|       616|
|  3|       15-64|indeterminado|       275|
|  4|       15-64|indeterminado|       275|
|  5|       20-59|       adulto|       244|
|  6|       15-74|indeterminado|       240|
|  7|       20-64|       adulto|       225|
|  8|       15-84|indeterminado|       187|
|  9|       15-69|indeterminado|       184|
| 10|       30-34|       adulto|        75|
| 11|       20-44|       adulto|        60|
| 12|       25-29|       adulto|        58|
| 13|       20-99|       adulto|        57|
| 14|        0-69|indeterminado|        54|
| 15|       35-39|       adulto|        46|
| 16|       18-61|indeterminado|        44|
| 17|       40-44|       adulto|        42|
| 18|        1-69|indeterminado|        37|
| 19|       19-59|indeterminado|        36|
| 20|        0-59|indeterminado|

In [28]:
monkeypoxAgedf1 = [{"id":"1","Faixa_etaria":"null","Classificacao":"null","Quantidade":66574},
                    {"id":"2","Faixa_etaria":"20-69","Classificacao":"adulto","Quantidade":616},
                    {"id":"3","Faixa_etaria":"15-64","Classificacao":"indeterminado","Quantidade":275},
                    {"id":"4","Faixa_etaria":"15-64","Classificacao":"indeterminado","Quantidade":275},
                    {"id":"5","Faixa_etaria":"20-59","Classificacao":"adulto","Quantidade":244},
                    {"id":"6","Faixa_etaria":"15-74","Classificacao":"indeterminado","Quantidade":240},
                    {"id":"7","Faixa_etaria":"20-64","Classificacao":"adulto","Quantidade":225},
                    {"id":"8","Faixa_etaria":"15-84","Classificacao":"indeterminado","Quantidade":187},
                    {"id":"9","Faixa_etaria":"15-69","Classificacao":"indeterminado","Quantidade":184},
                    {"id":"10","Faixa_etaria":"30-34","Classificacao":"adulto","Quantidade":75},
                    {"id":"11","Faixa_etaria":"20-44","Classificacao":"adulto","Quantidade":60},
                    {"id":"12","Faixa_etaria":"25-29","Classificacao":"adulto","Quantidade":58},
                    {"id":"13","Faixa_etaria":"20-99","Classificacao":"adulto","Quantidade":57},
                    {"id":"14","Faixa_etaria":"0-69","Classificacao":"indeterminado","Quantidade":54},
                    {"id":"15","Faixa_etaria":"35-39","Classificacao":"adulto","Quantidade":46},
                    {"id":"16","Faixa_etaria":"18-61","Classificacao":"indeterminado","Quantidade":44},
                    {"id":"17","Faixa_etaria":"40-44","Classificacao":"adulto","Quantidade":42},
                    {"id":"18","Faixa_etaria":"1-69","Classificacao":"indeterminado","Quantidade":37},
                    {"id":"19","Faixa_etaria":"19-59","Classificacao":"indeterminado","Quantidade":36},
                    {"id":"20","Faixa_etaria":"0-59","Classificacao":"indeterminado","Quantidade":34},
                    {"id":"21","Faixa_etaria":"23-50","Classificacao":"adulto","Quantidade":32}]

monkeypoxAgedf1 = spark.createDataFrame(monkeypoxAgedf1)

#get the sum of cost column groued by item
monkeypoxAgedf1.groupBy('Classificacao').sum('Quantidade').show(20)

spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-------------+---------------+
|Classificacao|sum(Quantidade)|
+-------------+---------------+
|indeterminado|           1366|
|       adulto|           1455|
|         null|          66574|
+-------------+---------------+



In [29]:
import plotly.express as px 
import numpy 
random_x = [1366,1455,66574] 
names = ['indeterminado','adulto','null'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

In [81]:
import plotly.express as px 
import numpy 
random_x = [1366,1455] 
names = ['indeterminado','adulto'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

##### Ranking dos sintomas mais registrados

In [30]:
monkeypoxdf.createOrReplaceTempView("MonkeypoxRank4")
spark.sql("select Symptoms, count(Country) as count_Country from MonkeypoxRank4 " +
          "group by Symptoms having count_Country >= 1 " + 
          "order by count_Country desc").show(10)

+--------------------+-------------+
|            Symptoms|count_Country|
+--------------------+-------------+
|                null|        69375|
|genital ulcer les...|           30|
|oral and genital ...|           17|
|  ulcerative lesions|           16|
|                Rash|           13|
|        skin lesions|            7|
|skin lesions, ulc...|            5|
|      vesicular rash|            5|
|      genital ulcers|            5|
| fever, skin lesions|            4|
+--------------------+-------------+
only showing top 10 rows



In [31]:
monkeypoxdf.createOrReplaceTempView("MonkeypoxRank5")
spark.sql("select Symptoms, count(Country) as count_Country from MonkeypoxRank5 " +
          "group by Symptoms having count_Country >= 1 " + 
          "order by count_Country desc").limit(10).toPandas()

,Symptoms,count_Country
0,None,69375
1,genital ulcer lesions,30
2,"oral and genital ulcers, fever",17
3,ulcerative lesions,16
4,Rash,13
5,skin lesions,7
6,"skin lesions, ulcerative lesions",5
7,vesicular rash,5
8,genital ulcers,5
9,"fever, skin lesions",4


In [32]:
monkeypoxSymptoms2=spark.createDataFrame([("1","None","69375"),("2","genital ulcer lesions","30"),("3","oral and genital ulcers, fever","17"),("4","ulcerative lesions","16"),("5","rash ","13"),("6","skin lesions","7"),("7","skin lesions, ulcerative lesions","5"), ("8","vesicular rash","5"),("9","genital ulcers","5"),("10","fever, skin lesions","4")],["ID","Sintomas","Total sintomas"])
monkeypoxSymptoms2.show()

monkeypoxSymptoms2=spark.createDataFrame([("1","None","69375"),("2","genital ulcer lesions","30"),("3","oral and genital ulcers, fever","17"),("4","ulcerative lesions","16"),("5","rash ","13"),("6","skin lesions","7"),("7","skin lesions, ulcerative lesions","5"), ("8","vesicular rash","5"),("9","genital ulcers","5"),("10","fever, skin lesions","4")],["ID","Sintomas","Total sintomas"])
monkeypoxSymptoms2.limit(10).toPandas()

+---+--------------------+--------------+
| ID|            Sintomas|Total sintomas|
+---+--------------------+--------------+
|  1|                None|         69375|
|  2|genital ulcer les...|            30|
|  3|oral and genital ...|            17|
|  4|  ulcerative lesions|            16|
|  5|               rash |            13|
|  6|        skin lesions|             7|
|  7|skin lesions, ulc...|             5|
|  8|      vesicular rash|             5|
|  9|      genital ulcers|             5|
| 10| fever, skin lesions|             4|
+---+--------------------+--------------+



,ID,Sintomas,Total sintomas
0,1,None,69375
1,2,genital ulcer lesions,30
2,3,"oral and genital ulcers, fever",17
3,4,ulcerative lesions,16
4,5,rash,13
5,6,skin lesions,7
6,7,"skin lesions, ulcerative lesions",5
7,8,vesicular rash,5
8,9,genital ulcers,5
9,10,"fever, skin lesions",4


In [33]:
#["ID","Sintoma","Grupo_Sintoma","Total_Grupo_Sintoma"]
monkeypoxSymptoms3=spark.createDataFrame([("1","None","None","69375"),("2","genital ulcer lesions","Genitália","30"),("3","oral ulcer","Boca","17"),("4","genital ulcers","Genitália","17"),("5","fever","Febre","17"),("6","ulcerative lesions","Pele","16"),("7","rash","Pele","13"),("8","skin lesions","Pele","7"),("9","skin lesions","Pele","5"),("10","ulcerative lesions","Pele","5"),("11","vesicular rash","Pele","5"),("12","genital ulcers", "Genitália","5"),("13","fever","Febre","4"),("14","skin lesions","Pele","4")],["ID","Sintoma","Grupo_Sintoma","Total_Grupo_Sintoma"])
monkeypoxSymptoms3.show()

monkeypoxSymptoms4=spark.createDataFrame([("1","None","None","69375"),("2","genital ulcer lesions","Genitália","30"),("3","oral ulcer","Boca","17"),("4","genital ulcers","Genitália","17"),("5","fever","Febre","17"),("6","ulcerative lesions","Pele","16"),("7","rash","Pele","13"),("8","skin lesions","Pele","7"),("9","skin lesions","Pele","5"),("10","ulcerative lesions","Pele","5"),("11","vesicular rash","Pele","5"),("12","genital ulcers", "Genitália","5"),("13","fever","Febre","4"),("14","skin lesions","Pele","4")],["ID","Sintoma","Grupo_Sintoma","Total_Grupo_Sintoma"])
monkeypoxSymptoms4.limit(14).toPandas()
#monkeypoxGender4=spark.createDataFrame([("67120","2422","5","15","31","1","1")],["Null","male","male ","Male","female","female ","Female"])
#monkeypoxGender4.show()

+---+--------------------+-------------+-------------------+
| ID|             Sintoma|Grupo_Sintoma|Total_Grupo_Sintoma|
+---+--------------------+-------------+-------------------+
|  1|                None|         None|              69375|
|  2|genital ulcer les...|    Genitália|                 30|
|  3|          oral ulcer|         Boca|                 17|
|  4|      genital ulcers|    Genitália|                 17|
|  5|               fever|        Febre|                 17|
|  6|  ulcerative lesions|         Pele|                 16|
|  7|                rash|         Pele|                 13|
|  8|        skin lesions|         Pele|                  7|
|  9|        skin lesions|         Pele|                  5|
| 10|  ulcerative lesions|         Pele|                  5|
| 11|      vesicular rash|         Pele|                  5|
| 12|      genital ulcers|    Genitália|                  5|
| 13|               fever|        Febre|                  4|
| 14|        skin lesion

,ID,Sintoma,Grupo_Sintoma,Total_Grupo_Sintoma
0,1,None,None,69375
1,2,genital ulcer lesions,Genitália,30
2,3,oral ulcer,Boca,17
3,4,genital ulcers,Genitália,17
4,5,fever,Febre,17
5,6,ulcerative lesions,Pele,16
6,7,rash,Pele,13
7,8,skin lesions,Pele,7
8,9,skin lesions,Pele,5
9,10,ulcerative lesions,Pele,5


In [34]:
import plotly.express as px 
import numpy 
random_x = [69375, 30, 17, 16, 13, 7, 5, 5, 5, 4] 
names = ['Null', 'Genital ulcer lesions', 'oral and genital ulcers + fever', 'ulcerative lesions', 'rash', 'skins lesions', 'skins lesions + ulcerative lesions', 'vesicular rash', 'genital ulcers', 'fever + skin lesions'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

In [35]:
import plotly.express as px 
import numpy 
random_x = [30, 17, 16, 13, 7, 5, 5, 5, 4] 
names = ['Genital ulcer lesions', 'oral and genital ulcers + fever', 'ulcerative lesions', 'rash', 'skins lesions', 'skins lesions + ulcerative lesions', 'vesicular rash', 'genital ulcers', 'fever + skin lesions'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

In [36]:
monkeypoxSymptomsdf =[{'id':1,'Sintoma':'None','Grupo_Sintoma':'None','Quantidade':	69375},
                    {'id':2,'Sintoma':'genital ulcer lesions','Grupo_Sintoma':'Genitália','Quantidade': 30},
                    {'id':3,'Sintoma':'oral ulcer','Grupo_Sintoma':'Boca','Quantidade':17},
                    {'id':4,'Sintoma':'genital ulcers','Grupo_Sintoma':'Genitália','Quantidade':17},
                    {'id':5,'Sintoma':'fever','Grupo_Sintoma':'Febre','Quantidade':17},
                    {'id':6,'Sintoma':'ulcerative lesions','Grupo_Sintoma':'Pele','Quantidade':16},
                    {'id':7,'Sintoma':'rash','Grupo_Sintoma':'Pele','Quantidade':13},
                    {'id':8,'Sintoma':'skin lesions','Grupo_Sintoma':'Pele','Quantidade':7},
                    {'id':9,'Sintoma':'skin lesions','Grupo_Sintoma':'Pele','Quantidade':5},
                    {'id':10,'Sintoma':'ulcerative lesions','Grupo_Sintoma':'Pele','Quantidade':5},
                    {'id':11,'Sintoma':'vesicular rash','Grupo_Sintoma':'Pele','Quantidade':5},
                    {'id':12,'Sintoma':'genital ulcers','Grupo_Sintoma':'Genitália','Quantidade':5},
                    {'id':13,'Sintoma':'fever','Grupo_Sintoma':'Febre','Quantidade':4},
                    {'id':14,'Sintoma':'skin lesions','Grupo_Sintoma':'Pele','Quantidade':4}]


monkeypoxSymptomsdf = spark.createDataFrame(monkeypoxSymptomsdf)

#get the sum of cost column groued by item
monkeypoxSymptomsdf.groupBy('Grupo_Sintoma').sum('Quantidade').show()

spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning:

inferring schema from dict is deprecated,please use pyspark.sql.Row instead



+-------------+---------------+
|Grupo_Sintoma|sum(Quantidade)|
+-------------+---------------+
|         None|          69375|
|    Genitália|             52|
|         Pele|             55|
|        Febre|             21|
|         Boca|             17|
+-------------+---------------+



In [37]:
import plotly.express as px 
import numpy 
random_x = [69375, 52,55,21,17] 
names = ['None','Genitália','Pele','Febre','Boca'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

In [38]:
import plotly.express as px 
import numpy 
random_x = [52,55,21,17] 
names = ['Genitália','Pele','Febre','Boca'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

##### Ranking do Sexo de pacientes com casos registrados (no estado da arte em que se encontra o dataframe)

In [39]:
monkeypoxdf.groupby('Gender').count().show()

+-------+-----+
| Gender|count|
+-------+-----+
|   null|67164|
| Female|    1|
| female|   31|
|  male |    5|
|   male| 2422|
|   Male|   15|
|female |    1|
+-------+-----+



In [40]:
monkeypoxdf.createOrReplaceTempView("MonkeypoxRank3")
spark.sql("select Gender, count(Country) as count_Country from MonkeypoxRank3 " +
          "group by Gender having count_Country >= 1 " + 
          "order by count_Country desc").show()

+-------+-------------+
| Gender|count_Country|
+-------+-------------+
|   null|        67120|
|   male|         2422|
| female|           31|
|   Male|           15|
|  male |            5|
| Female|            1|
|female |            1|
+-------+-------------+



In [41]:
monkeypoxdf.groupBy("Gender").agg(countDistinct('Country')) \
    .show(truncate=False)

+-------+-----------------------+
|Gender |count(DISTINCT Country)|
+-------+-----------------------+
|null   |111                    |
|Female |1                      |
|female |19                     |
|male   |4                      |
|male   |78                     |
|Male   |3                      |
|female |1                      |
+-------+-----------------------+



In [42]:
from pandas.core.sorting import get_indexer_dict
#Tranformando em TempView com correção de duplicidade + soma dos Sexos registrados no Dataframe
monkeypoxdf.createOrReplaceTempView("MonkeypoxRank3")
#spark.sql(
#    """select Gender, count(Country) as count_Country from MonkeypoxRank3 """ +
#    """group by Gender having count_Country >= 1 """ + 
#    """order by count_Country desc"""    
#    ).show()
spark.sql(
"""SELECT Gender,
    Count(Country) AS total_Sexo
  FROM MonkeypoxRank3
  GROUP BY Gender
  ORDER BY Count(Country) DESC"""
).show()

#df.groupBy("state") \
#  .agg(sum("salary").alias("sum_salary")) \
#  .filter(col("sum_salary") > 100000)  \
#  .sort(desc("sum_salary")) \
#  .show()




+-------+----------+
| Gender|total_Sexo|
+-------+----------+
|   null|     67120|
|   male|      2422|
| female|        31|
|   Male|        15|
|  male |         5|
| Female|         1|
|female |         1|
+-------+----------+



In [43]:
monkeypoxGender1=spark.createDataFrame([("1","null","67120"),("2","male","2422"),("3","female","31"),("4","Male","15"),("5","male ","5"),("6","Female","1"),("7","female","1")],["ID","Sexo","Parcial"])
monkeypoxGender1.show()

monkeypoxGender2=spark.createDataFrame([("1","null","67120"),("2","male","2422"),("3","female","31"),("4","Male","15"),("5","male ","5"),("6","Female","1"),("7","female","1")],["ID","Sexo","Parcial"])
monkeypoxGender2.show()

monkeypoxGender3=spark.createDataFrame([("1","67120","null","null"),("2","null","2422","null"),("3","null","null","31"),("4","null","15","null"),("5","null","5","null"),("6","null","null","1"),("7","null","null","1")],["ID","Null","Masculino","Feminino"])
monkeypoxGender3.show()

monkeypoxGender4=spark.createDataFrame([("67120","2422","5","15","31","1","1")],["Null","male","male ","Male","female","female ","Female"])
monkeypoxGender4.show()

#df1.join(df2,df1.col1 == df2.col1,'inner').select(df1.col1,df2.col2,df1.col3,df1.col2,df2.col4).withColumn("col3",(df1.col3 / df2.col2).cast("double")).withColumn("col2",(df1.col2 + df2.col2).cast("int")).show()

+---+------+-------+
| ID|  Sexo|Parcial|
+---+------+-------+
|  1|  null|  67120|
|  2|  male|   2422|
|  3|female|     31|
|  4|  Male|     15|
|  5| male |      5|
|  6|Female|      1|
|  7|female|      1|
+---+------+-------+

+---+------+-------+
| ID|  Sexo|Parcial|
+---+------+-------+
|  1|  null|  67120|
|  2|  male|   2422|
|  3|female|     31|
|  4|  Male|     15|
|  5| male |      5|
|  6|Female|      1|
|  7|female|      1|
+---+------+-------+

+---+-----+---------+--------+
| ID| Null|Masculino|Feminino|
+---+-----+---------+--------+
|  1|67120|     null|    null|
|  2| null|     2422|    null|
|  3| null|     null|      31|
|  4| null|       15|    null|
|  5| null|        5|    null|
|  6| null|     null|       1|
|  7| null|     null|       1|
+---+-----+---------+--------+

+-----+----+-----+----+------+-------+------+
| Null|male|male |Male|female|female |Female|
+-----+----+-----+----+------+-------+------+
|67120|2422|    5|  15|    31|      1|     1|
+-----+---

In [44]:
monkeypoxGender5=spark.createDataFrame([("67120","2442","33")],["Null","male","female"])
monkeypoxGender5.show()

+-----+----+------+
| Null|male|female|
+-----+----+------+
|67120|2442|    33|
+-----+----+------+



In [45]:
import plotly.express as px 
import numpy 
random_x = [67120, 2442, 33] 
names = ['Null', 'male', 'female'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

In [46]:
import plotly.express as px 
import numpy 
random_x = [2442, 33] 
names = ['male', 'female'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()

#### Explorando possíveis storytellings

In [47]:
from pyspark.sql.functions import datediff,col
 
monkeypoxDatedif = monkeypoxdf.withColumn("Diferenca_em_dias", datediff(col("Date_confirmation"),col("Date_onset"))) \
    .drop("Symptoms","Hospitalised (Y/N/NA)","Isolated (Y/N/NA)","Travel_history (Y/N/NA)","Travel_history_location", \
          "Travel_history_country","Date_hospitalisation","Date_isolation","Outcome","Contact_comment", \
          "Contact_ID","Contact_location","Travel_history_entry","Travel_history_start", \
          "Genomics_Metadata","Confirmation_method","Source","Source_II","Source_III", \
          "Source_IV","Source_V","Source_VI","Source_VII","Date_entry","Date_death","Date_last_modified").show(truncate=False)
monkeypoxDatedif

#monkeypoxDatedif.na.drop(subset=["Diferenca_em_dias"]) \
#   .show(truncate=False)

+---+---------+-----------------------------------+---------+--------+------------+-----+------+-------------------+-------------------+-----------------+
|ID |Status   |Location                           |City     |Country |Country_ISO3|Age  |Gender|Date_onset         |Date_confirmation  |Diferenca_em_dias|
+---+---------+-----------------------------------+---------+--------+------------+-----+------+-------------------+-------------------+-----------------+
|N1 |confirmed|Guy's and St Thomas Hospital London|London   |England |GBR         |null |null  |2022-04-29 00:00:00|2022-05-06 00:00:00|7                |
|N2 |confirmed|Guy's and St Thomas Hospital London|London   |England |GBR         |null |null  |2022-05-05 00:00:00|2022-05-12 00:00:00|7                |
|N3 |confirmed|London                             |London   |England |GBR         |null |null  |2022-04-30 00:00:00|2022-05-13 00:00:00|13               |
|N4 |confirmed|London                             |London   |England |

In [48]:
newmonkeypoxdf1 = monkeypoxdf.withColumnRenamed("Location", "Localizacao") \
              .withColumnRenamed("City", "Cidade") \
              .withColumnRenamed("Country", "Pais") \
              .withColumnRenamed("Country_ISO3", "Cod_ISO3") \
              .withColumnRenamed("Age", "Idade") \
              .withColumnRenamed("Gender", "Sexo") \
              .withColumnRenamed("Date_onset", "Data_entrada_sistema") \
              .withColumnRenamed("Date_confirmation", "Data_confirmacao") \
              .withColumnRenamed("Symptoms", "Sintomas") \
              .withColumnRenamed("Hospitalised (Y/N/NA)", "Hospitalizado") \
              .withColumnRenamed("Date_hospitalisation", "Data_internacao") \
              .withColumnRenamed("Isolated (Y/N/NA)", "Em_isolamento") \
              .withColumnRenamed("Date_isolation", "Data_isolamento") \
              .withColumnRenamed("Outcome", "Detectado") \
              .withColumnRenamed("Contact_comment", "Comentarios_contato") \
              .withColumnRenamed("Contact_ID", "Identidade_Contato") \
              .withColumnRenamed("Contact_location", "Cidade_contato") \
              .withColumnRenamed("Travel_history (Y/N/NA)", "Viajou") \
              .withColumnRenamed("Travel_history_entry", "Data_viagem") \
              .withColumnRenamed("Travel_history_start", "Data_inicio_viagem") \
              .withColumnRenamed("Travel_history_location", "Localidade_visitada") \
              .withColumnRenamed("Travel_history_country", "Pais_visitado") \
              .withColumnRenamed("Genomics_Metadata", "Genoma_virus") \
              .withColumnRenamed("Confirmation_method", "Metodo_confirmacao") \
              .withColumnRenamed("Source", "Fonte") \
              .withColumnRenamed("Source_II", "Fonte_II") \
              .withColumnRenamed("Source_III", "Fonte_III") \
              .withColumnRenamed("Source_IV", "Fonte_IV") \
              .withColumnRenamed("Source_V", "Fonte_V") \
              .withColumnRenamed("Source_VI", "Fonte_VI") \
              .withColumnRenamed("Source_VII", "Fonte_VII") \
              .withColumnRenamed("Date_entry", "Data_registro") \
              .withColumnRenamed("Date_death", "Data_obito") \
              .withColumnRenamed("Date_last_modified", "Data_ultima_atualizacao")
newmonkeypoxdf1.printSchema() 
newmonkeypoxdf1.show()

root
 |-- ID: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Localizacao: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Cod_ISO3: string (nullable = true)
 |-- Idade: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Data_entrada_sistema: timestamp (nullable = true)
 |-- Data_confirmacao: timestamp (nullable = true)
 |-- Sintomas: string (nullable = true)
 |-- Hospitalizado: string (nullable = true)
 |-- Data_internacao: timestamp (nullable = true)
 |-- Em_isolamento: string (nullable = true)
 |-- Data_isolamento: timestamp (nullable = true)
 |-- Detectado: string (nullable = true)
 |-- Comentarios_contato: string (nullable = true)
 |-- Identidade_Contato: integer (nullable = true)
 |-- Cidade_contato: string (nullable = true)
 |-- Viajou: string (nullable = true)
 |-- Data_viagem: string (nullable = true)
 |-- Data_inicio_viagem: string (nullable = true)
 |-- Localidade_visitada: string (nul

Explorando a coluna "Gender""

In [49]:
monkeypoxdf4 = monkeypoxdf.select(countDistinct("Gender"))
monkeypoxdf4.show(truncate=False)
print("Distinct Count of Gender: "+str(monkeypoxdf4.collect()[0][0]))

+----------------------+
|count(DISTINCT Gender)|
+----------------------+
|6                     |
+----------------------+

Distinct Count of Gender: 6


In [50]:
NewMonkeyPox1 = newmonkeypoxdf1.createOrReplaceTempView("NewMonkeyPox1")
spark.sql("select Status, Pais, CASE WHEN Sexo = 'Female' THEN 'female' " +
          "WHEN Sexo = 'female ' THEN 'female' " + 
          "WHEN Sexo = 'Male' THEN 'male' " +
          "WHEN Sexo = 'male ' THEN 'male' WHEN Sexo IS NULL THEN '' " +
          "ELSE Sexo END as novo_sexo from NewMonkeyPox1").distinct().count()
NewMonkeyPox1

In [51]:
from pyspark.sql.functions import when
monkeypoxdfGender = monkeypoxdf.withColumn('Sexo', 
    when(monkeypoxdf.Gender.endswith('male '),regexp_replace(monkeypoxdf.Gender,'male ','male')) \
   .when(monkeypoxdf.Gender.endswith('Male'),regexp_replace(monkeypoxdf.Gender,'Male','male')) \
   .when(monkeypoxdf.Gender.endswith('female '),regexp_replace(monkeypoxdf.Gender,'female ','female')) \
   .when(monkeypoxdf.Gender.endswith('Female'),regexp_replace(monkeypoxdf.Gender,'Female','female')) \
   .otherwise(monkeypoxdf.Gender)) \
   .show(truncate=False)
monkeypoxdfGender

+---+---------+-----------------------------------+---------+--------+------------+-----+------+-------------------+-------------------+--------------------------------+---------------------+--------------------+-----------------+-------------------+-------+-------------------------------+----------+----------------+-----------------------+--------------------+--------------------+-----------------------+----------------------+------------------+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+----------+---------+--------+---------+----------+-------------------+----------+-------------------+----+
|ID |Status   |Location                           |City     |Country |Country_ISO3|Age  |Gender|Date_onset         |Date_confirmation  |Symptom

In [52]:
spark.sql("select DISTINCT Pais, CASE WHEN Sexo = 'Female' THEN 'female' " +
          "WHEN Sexo = 'female ' THEN 'female' " + 
          "WHEN Sexo = 'Male' THEN 'male' " +
          "WHEN Sexo = 'male ' THEN 'male' WHEN Sexo IS NULL THEN '' " +
          "ELSE Sexo END as novo_sexo from NewMonkeyPox1").show()

+------------------+---------+
|              Pais|novo_sexo|
+------------------+---------+
|           Bahamas|     male|
|            Monaco|         |
|           Bahamas|         |
|            Guyana|     male|
|       New Zealand|         |
|            Taiwan|         |
|           Ukraine|     male|
|            Guyana|         |
|            Belize|         |
|           England|         |
|            Greece|     male|
|            Israel|     male|
|           Morocco|         |
|         Greenland|         |
|             Spain|         |
|          Thailand|         |
|Dominican Republic|     male|
|           Lebanon|         |
|           Romania|         |
|           Liberia|     male|
+------------------+---------+
only showing top 20 rows



In [53]:
from pyspark.sql.functions import expr
monkeypoxdf2=monkeypoxdf.withColumn("Gender", expr("CASE WHEN Gender = 'Male' THEN 'male' " +
           "WHEN Gender = 'male ' THEN 'male'" +
           "WHEN Gender = 'Female' THEN 'female'" +
           "WHEN Gender = 'female ' THEN 'female' END"))
monkeypoxdf2.show()

+---+---------+--------------------+---------+--------+------------+-----+------+-------------------+-------------------+--------------------+---------------------+--------------------+-----------------+-------------------+-------+--------------------+----------+----------------+-----------------------+--------------------+--------------------+-----------------------+----------------------+------------------+-------------------+--------------------+--------------------+----------+---------+--------+---------+----------+-------------------+----------+-------------------+
| ID|   Status|            Location|     City| Country|Country_ISO3|  Age|Gender|         Date_onset|  Date_confirmation|            Symptoms|Hospitalised (Y/N/NA)|Date_hospitalisation|Isolated (Y/N/NA)|     Date_isolation|Outcome|     Contact_comment|Contact_ID|Contact_location|Travel_history (Y/N/NA)|Travel_history_entry|Travel_history_start|Travel_history_location|Travel_history_country| Genomics_Metadata|Confirmatio

In [54]:
from pyspark.sql.functions import approx_count_distinct

print("Total de faixa etárias distintas: " + \
     str(monkeypoxdf.select(approx_count_distinct("Age")).collect()[0][0]))

Total de faixa etárias distintas: 95


In [55]:
monkeypoxdf.select(collect_list("Age")).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Contagem de valores vazios em cada coluna do dataframe

In [56]:
from pyspark.sql.functions import isnan, when, count, col
newmonkeypoxdf_Null=["Status", "Localizacao", "Cidade", "Pais", "Cod_ISO3","Idade", "Sexo", "Sintomas","Hospitalizado","Em_isolamento","Detectado","Comentarios_contato","Identidade_Contato","Cidade_contato", "Viajou", "Data_viagem","Data_inicio_viagem","Localidade_visitada","Pais_visitado","Genoma_virus", "Metodo_confirmacao", "Fonte", "Fonte_II", "Fonte_III", "Fonte_IV", "Fonte_V", "Fonte_VI", "Fonte_VII"]
newmonkeypoxdf1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in newmonkeypoxdf_Null]).show()

+------+-----------+------+----+--------+-----+-----+--------+-------------+-------------+---------+-------------------+------------------+--------------+------+-----------+------------------+-------------------+-------------+------------+------------------+-----+--------+---------+--------+-------+--------+---------+
|Status|Localizacao|Cidade|Pais|Cod_ISO3|Idade| Sexo|Sintomas|Hospitalizado|Em_isolamento|Detectado|Comentarios_contato|Identidade_Contato|Cidade_contato|Viajou|Data_viagem|Data_inicio_viagem|Localidade_visitada|Pais_visitado|Genoma_virus|Metodo_confirmacao|Fonte|Fonte_II|Fonte_III|Fonte_IV|Fonte_V|Fonte_VI|Fonte_VII|
+------+-----------+------+----+--------+-----+-----+--------+-------------+-------------+---------+-------------------+------------------+--------------+------+-----------+------------------+-------------------+-------------+------------+------------------+-----+--------+---------+--------+-------+--------+---------+
|    44|      17252| 68256|  44|      44

In [57]:
spark.sql('select * from NewMonkeyPox1 where isNULL(Sexo)').count()

67164

In [58]:
monkeypoxdf.createOrReplaceTempView("monkeypoxnulldf")
monkeypoxnull = spark.sql('select Status, Age, Gender,  Symptoms from monkeypoxnulldf where isNull(Age) and isNull(Gender) and isNull(Symptoms)')
monkeypoxnull.count()
monkeypoxnull.show()

+---------+----+------+--------+
|   Status| Age|Gender|Symptoms|
+---------+----+------+--------+
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|discarded|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
|confirmed|null|  null|    null|
+---------+----+------+--------+
only showing top 20 rows



In [ ]:
#spark.sql(
#"""SELECT Status, Age, Gender, Symptoms from monkeypoxnulldf where isNull(Age) and isNull(Gender)
#    Count(Country) AS total_Sexo
#  FROM MonkeypoxRank3
#  GROUP BY Gender
#  ORDER BY Count(Country) DESC"""
#).show()

In [59]:
from pyspark.sql.functions import count
monkeypoxdf.groupBy("Status") \
  .agg(count("Country").alias("Avg_Country")) \
  .sort(col("Avg_Country").desc()) \
  .show()

+----------+-----------+
|    Status|Avg_Country|
+----------+-----------+
| confirmed|      65215|
| suspected|       3673|
| discarded|        375|
|omit_error|        332|
|      null|          0|
+----------+-----------+



In [60]:
monkeypoxnullnumber = monkeypoxnull.filter(monkeypoxnull.Status == "confirmed")
monkeypoxnullnumber.count()

61636

In [61]:
monkeypoxdf.createOrReplaceTempView("mpnullBrazil")
mpnullBrazildf = spark.sql('select Status, Country, Age, Gender, Symptoms from mpnullBrazil where isNull(Age) and isNull(Gender) and isNull(Symptoms) and Country =="Brazil" and Status =="confirmed"')
mpnullBrazildf.show()

+---------+-------+----+------+--------+
|   Status|Country| Age|Gender|Symptoms|
+---------+-------+----+------+--------+
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
+---------+-------+----+------+--------+
only showing top

##### Dados da MonkeyPox no Brasil

In [63]:
mpnullBrazildf.show()

+---------+-------+----+------+--------+
|   Status|Country| Age|Gender|Symptoms|
+---------+-------+----+------+--------+
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
|confirmed| Brazil|null|  null|    null|
+---------+-------+----+------+--------+
only showing top

In [64]:
mpnullBrazildf2 = mpnullBrazildf.filter(mpnullBrazildf.Country == "Brazil") and mpnullBrazildf.filter(mpnullBrazildf.Status == "confirmed")
mpnullBrazildf2.count()

7269

In [65]:
mpnullBrazildf3 = spark.sql('select Status, Pais, Idade, Sexo, Sintomas from NewMonkeyPox1 where isNULL(Sexo) and Status == "confirmed" and Pais == "Brazil"').count()
mpnullBrazildf3

7277

In [66]:
mpnullBrazildf4 = spark.sql('select Status, Pais, Idade, Sexo, Sintomas from NewMonkeyPox1 where Sexo == "male" and Status == "confirmed" and Pais =="Brazil"').count()
mpnullBrazildf4

23

In [67]:
mpnullBrazildf5 = spark.sql('select Status, Pais, Idade, Sexo, Sintomas from NewMonkeyPox1 where Sexo == "female" and Status == "confirmed" and Pais =="Brazil"').count()
mpnullBrazildf5

0

In [68]:
mpIdadedf = spark.sql(
    """select distinct Idade, Status, Sintomas from NewMonkeyPox1 where Status like 'confirmed'"""
).count()
mpIdadedf

195

#### Pivotando colunas-chave do dataset

In [69]:
monkeypoxdf.groupBy("Country").pivot("Age").count().show()

+-----------+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|    Country|null|0-14|0-29|0-39|0-49| 0-5|0-59|0-69| 0-9|1-69|10-14|15-19|15-20|15-39|15-59|15-64|15-69|15-74|15-84|18-61|19-24|19-59|20-24|20-25|20-29|20-35|20-39|20-40|20-44|20-46|20-49|20-50|20-54|20-59|20-60|20-62|20-64|20-65|20-69|20-99|21-54|22-43|22-55|22-59|23-46|23-50|24-29|25-29|25-30|25-34|25-39|25-44|25-49|25-52|25-53|26-30|26-54|28-43|29-41|29-43|30-34|30-35|30-39|30-40|30-49|30-50|30-54|30-59|30-60|31-4

In [70]:
monkeypoxdf.groupBy("Age").pivot("Country").count().show()

+-----+----+-------+---------+-----+---------+-------+-------+-------+----------+--------+-------+------+-----+-------+-------+----------------------+------+--------+--------+--------+------+--------------+------------------------+-----+-----+--------+----------+-------+----+-------+------+--------------+--------------------------------+-------+------------------+-------+-----+-----------+-------+-------+----+-------+------+-------------+-------+-------+-----+---------+------+---------+----------+----+---------+------+-----+--------+---------+-------+-------+-----+---------+----+-------+------+-----+-------+-----+------+------+------+-------+-------+---------+----------+------+--------+-----+----------+---------+------+-------+------+----------+-------+-----+-----------+-------------+-----------+-------+----------------+------+--------+------+--------+----+-----------+------+--------+-----------+-----+-----------------+-------+------+--------------------------+------------+--------+---

In [71]:
monkeypoxdf.groupBy("Gender").pivot("Country").count().show()

+-------+----+-------+---------+-----+---------+-------+-------+-------+----------+--------+-------+------+-----+-------+-------+----------------------+------+--------+--------+--------+------+--------------+------------------------+-----+-----+--------+----------+-------+----+-------+------+--------------+--------------------------------+-------+------------------+-------+-----+-----------+-------+-------+----+-------+------+-------------+-------+-------+-----+---------+------+---------+----------+----+---------+------+-----+--------+---------+-------+-------+-----+---------+----+-------+------+-----+-------+-----+------+------+------+-------+-------+---------+----------+------+--------+-----+----------+---------+------+-------+------+----------+-------+-----+-----------+-------------+-----------+-------+----------------+------+--------+------+--------+----+-----------+------+--------+-----------+-----+-----------------+-------+------+--------------------------+------------+--------+-

In [72]:
monkeypoxPivot = monkeypoxdf.groupBy("Country").pivot("Gender").count().show()
monkeypoxPivot

#monkeypoxPivot = monkeypoxdf.groupBy("Country").pivot("Gender").count().show()
#monkeypoxPivot

+-----------+----+------+----+------+-------+----+-----+
|    Country|null|Female|Male|female|female |male|male |
+-----------+----+------+----+------+-------+----+-----+
|   Paraguay|   2|  null|null|  null|   null|   1| null|
|     Russia|null|  null|null|  null|   null|   2| null|
|     Sweden| 121|  null|null|  null|   null|  65| null|
|     Guyana|   1|  null|null|  null|   null|   1| null|
|Philippines|   3|  null|null|  null|   null|   1| null|
|   Malaysia|   3|  null|null|  null|   null|null| null|
|  Singapore|   4|  null|null|  null|   null|  15| null|
|       Fiji|   1|  null|null|  null|   null|null| null|
|     Turkey|  14|  null|null|  null|   null|   1| null|
|     Malawi|   1|  null|null|  null|   null|null| null|
|    Germany|3484|  null|null|  null|      1| 103|    2|
|   Cambodia|  11|  null|null|  null|   null|null| null|
|     Jordan|null|  null|null|  null|   null|   1| null|
|      Sudan|   7|  null|null|  null|   null|null| null|
|     France|3778|  null|null| 

In [73]:
monkeypoxPivotSG = monkeypoxdf.groupBy("Symptoms").pivot("Gender").count().show()
monkeypoxPivotSG

+--------------------+-----+------+----+------+-------+----+-----+
|            Symptoms| null|Female|Male|female|female |male|male |
+--------------------+-----+------+----+------+-------+----+-----+
|headache, skin le...|    2|  null|null|  null|   null|null| null|
|Perianal rash, fever| null|  null|null|  null|   null|   1| null|
|Fever, chills, fa...|    1|  null|null|  null|   null|null| null|
|headache, muscle ...| null|  null|null|  null|   null|   1| null|
|fever, general ma...| null|  null|null|  null|   null|   1| null|
|fever, outbreak o...| null|  null|null|     1|   null|null| null|
|                rash|    2|  null|null|  null|   null|   1| null|
|rash, body pains,...| null|  null|null|  null|   null|   1| null|
|             lesions| null|  null|null|  null|   null|   1| null|
|genital ulcer les...| null|  null|null|  null|   null|  30| null|
|characteristic sy...| null|  null|null|  null|   null|   1| null|
| skin manifestations|    1|  null|null|  null|   null|null| n

In [74]:
monkeypoxPivotSA = monkeypoxdf.groupBy("Symptoms").pivot("Age").count().show()
monkeypoxPivotSA

+--------------------+-----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|            Symptoms| null|0-14|0-29|0-39|0-49| 0-5|0-59|0-69| 0-9|1-69|10-14|15-19|15-20|15-39|15-59|15-64|15-69|15-74|15-84|18-61|19-24|19-59|20-24|20-25|20-29|20-35|20-39|20-40|20-44|20-46|20-49|20-50|20-54|20-59|20-60|20-62|20-64|20-65|20-69|20-99|21-54|22-43|22-55|22-59|23-46|23-50|24-29|25-29|25-30|25-34|25-39|25-44|25-49|25-52|25-53|26-30|26-54|28-43|29-41|29-43|30-34|30-35|30-39|30-40|30-49|30-50|30

In [75]:
#from pyspark.sql.functions import col,lit,create_map
df2 = monkeypoxdf.withColumn("Sexo+Idade",create_map(
        lit("Gender"),col("Gender"),
        lit("Age"),col("Age")
        )).drop("Gender","Age") \
        .withColumn("Sintomas+Hospitalizado+Isolado",create_map(
        lit("Symptoms"),col("Symptoms"),
        lit("Hospitalised (Y/N/NA)"),col("Hospitalised (Y/N/NA)"),
        lit("Isolated (Y/N/NA)"),col("Isolated (Y/N/NA)")
        )).drop("Symptoms","Hospitalised (Y/N/NA)","Isolated (Y/N/NA)") \
        .withColumn("Viajou?+Local_visitado",create_map(
            lit("Travel_history (Y/N/NA)"), col("Travel_history (Y/N/NA)"),
            lit("Travel_history_location"), col("Travel_history_location"),
            lit("Travel_history_country"), col("Travel_history_country")
        )).drop("Country_ISO3","Date_onset","Travel_history (Y/N/NA)","Travel_history_location","Travel_history_country") \
        .drop("Date_hospitalisation","Date_isolation","Outcome","Contact_comment","Contact_ID","Contact_location","Travel_history_entry","Travel_history_start", \
              "Genomics_Metadata","Confirmation_method","Source","Source_II","Source_III","Source_IV","Source_V","Source_VI","Source_VII","Date_entry","Date_death","Date_last_modified")
df2.printSchema()
df2.show(truncate=False)

root
 |-- ID: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Date_confirmation: timestamp (nullable = true)
 |-- Sexo+Idade: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- Sintomas+Hospitalizado+Isolado: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- Viajou?+Local_visitado: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---+---------+-----------------------------------+---------+--------+-------------------+------------------------------+------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|ID |Status   |Location                         

## Storytelling: O peso do vazio

In [87]:
newmonkeypoxdf_Null=["Status", "Localizacao", "Cidade", "Pais", "Cod_ISO3","Idade", "Sexo", "Sintomas","Hospitalizado","Em_isolamento","Detectado","Comentarios_contato","Identidade_Contato","Cidade_contato", "Viajou", "Data_viagem","Data_inicio_viagem","Localidade_visitada","Pais_visitado","Genoma_virus", "Metodo_confirmacao", "Fonte", "Fonte_II", "Fonte_III", "Fonte_IV", "Fonte_V", "Fonte_VI", "Fonte_VII"]
newmonkeypoxdf1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in newmonkeypoxdf_Null]).show()

+------+-----------+------+----+--------+-----+-----+--------+-------------+-------------+---------+-------------------+------------------+--------------+------+-----------+------------------+-------------------+-------------+------------+------------------+-----+--------+---------+--------+-------+--------+---------+
|Status|Localizacao|Cidade|Pais|Cod_ISO3|Idade| Sexo|Sintomas|Hospitalizado|Em_isolamento|Detectado|Comentarios_contato|Identidade_Contato|Cidade_contato|Viajou|Data_viagem|Data_inicio_viagem|Localidade_visitada|Pais_visitado|Genoma_virus|Metodo_confirmacao|Fonte|Fonte_II|Fonte_III|Fonte_IV|Fonte_V|Fonte_VI|Fonte_VII|
+------+-----------+------+----+--------+-----+-----+--------+-------------+-------------+---------+-------------------+------------------+--------------+------+-----------+------------------+-------------------+-------------+------------+------------------+-----+--------+---------+--------+-------+--------+---------+
|    44|      17252| 68256|  44|      44

In [79]:
px.bar(x=["Status","Localizacao","Cidade","Pais","Cod_ISO3","Idade","Sexo","Sintomas","Hospitalizado","Em_isolamento","Detectado","Comentarios_contato","Identidade_Contato","Cidade_Contato","Viajou","Data_viagem","Data_inicio_viagem","Localidade_visitada","Pais_visitado","Genoma_virus", "Metodo_confirmacao", "Fonte", "Fonte_II", "Fonte_III", "Fonte_IV", "Fonte_V", "Fonte_VI", "Fonte_VII"], y=[44,17252,8256,44,44,66618,67164,69419,69285,69146,69538,69548,69612,69633,69274,69597,69628,69525,69540,69615,69539,44,61442,68746,69585,69639,69639,69639])

In [78]:
px.line(x=["Status","Localizacao","Cidade","Pais","Cod_ISO3","Idade","Sexo","Sintomas","Hospitalizado","Em_isolamento","Detectado","Comentarios_contato","Identidade_Contato","Cidade_Contato","Viajou","Data_viagem","Data_inicio_viagem","Localidade_visitada","Pais_visitado","Genoma_virus", "Metodo_confirmacao", "Fonte", "Fonte_II", "Fonte_III", "Fonte_IV", "Fonte_V", "Fonte_VI", "Fonte_VII"], y=[44,17252,8256,44,44,66618,67164,69419,69285,69146,69538,69548,69612,69633,69274,69597,69628,69525,69540,69615,69539,44,61442,68746,69585,69639,69639,69639])

In [89]:
newmonkeypoxdf_Null1=["Status", "Localizacao", "Cidade", "Pais", "Cod_ISO3","Idade", "Sexo", "Sintomas","Hospitalizado","Viajou"]
newmonkeypoxdf1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in newmonkeypoxdf_Null1]).show()

+------+-----------+------+----+--------+-----+-----+--------+-------------+------+
|Status|Localizacao|Cidade|Pais|Cod_ISO3|Idade| Sexo|Sintomas|Hospitalizado|Viajou|
+------+-----------+------+----+--------+-----+-----+--------+-------------+------+
|    44|      17252| 68256|  44|      44|66618|67164|   69419|        69285| 69274|
+------+-----------+------+----+--------+-----+-----+--------+-------------+------+



In [90]:
px.line(x=["Status", "Localizacao", "Cidade", "Pais", "Cod_ISO3","Idade", "Sexo", "Sintomas","Hospitalizado","Viajou"], y=[44,17252,68256,44,44,66618,67164,69419,69285,69274])

In [91]:
import plotly.express as px 
import numpy 
random_x = [44,17252,68256,44,44,66618,67164,69419,69285,69274] 
names = ["Status", "Localizacao", "Cidade", "Pais", "Cod_ISO3","Idade", "Sexo", "Sintomas","Hospitalizado","Viajou"]
  
fig = px.pie(values=random_x, names=names) 
fig.show()

In [77]:
import plotly.express as px 
import numpy 
random_x = [44,17252,68256,44,44,66618,67164,69419] 
names = ['Status','Localizacao','Cidade','Pais','Cod_ISO3','Idade','Sexo','Sintomas'] 
  
fig = px.pie(values=random_x, names=names) 
fig.show()